<div class="alert alert-block alert-success">
FASE 1: CARGA DE DATOS, TRATAMIENTO, LIMPIADO Y CREACIÓN DE DATAFRAME FINAL
<br><br>FASE 2: ANÁLISIS VARIABLES, FEATURES SELECTION
<br><br>FASE 3: DEFINICIÓN FUNCIONES MACHINE LEARNING Y APLICACIÓN DE MODELOS
<br><br>FASE 4: CONCLUSIONES
</div>

<div class="alert alert-block alert-success">
FASE 1
</DIV>

<div class="alert alert-block alert-info"> 
LIBRERIAS
</div>

In [1]:
import pandas as pd
import numpy as np
import time, datetime
import math
import warnings
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import time

# SPLIT + NORMALIZADO
#from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

#METRICAS
from sklearn import metrics 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# REGRESORES
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.datasets import load_boston, load_iris
#from sklearn.linear_model import LinearRegression, Ridge
#from sklearn import svm

# AMBOS
import xgboost as xgb

# CLASIFICACION
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

min_max_scaler = MinMaxScaler()

# FEATURES SELECTION
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [2]:
%pylab inline
%matplotlib inline
style.use('ggplot')
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


<div class="alert alert-block alert-info">
CARGA DE DATOS
</div>

In [3]:
# FICHEROS ALOJADOS EN UN SERVIDOR NUESTRO: http://dreamlife.es
ruta = "http://dreamlife.es/ripple.csv"
dfTransacciones = pd.read_csv(ruta,delimiter=";", decimal=",")

ruta = "http://dreamlife.es/ballenas.csv"
dfTransBallenas = pd.read_csv(ruta,delimiter=";", decimal=",")

ruta = "http://dreamlife.es/tweets.csv"
dfTweets = pd.read_csv(ruta,delimiter=";", decimal=",")

<div class="alert alert-block alert-info">
TRATAMIENTO DE DATOS
</div> 

In [4]:
# TRANSFORM
dfTransacciones["FECHA"] = pd.to_datetime(dfTransacciones["FECHA"])
dfTransacciones.index = pd.to_datetime(dfTransacciones["FECHA"], unit='s')
dfTransacciones = dfTransacciones.sort_values('FECHA')

dfTransBallenas["FECHA"] = dfTransBallenas["FECHA_OPERACION"]
dfTransBallenas["FECHA"] = pd.to_datetime(dfTransBallenas["FECHA"])
dfTransBallenas.index = pd.to_datetime(dfTransBallenas["FECHA"], unit='s')
dfTransBallenas = dfTransBallenas.sort_values('FECHA')

dfTweets["FECHA"] = pd.to_datetime(dfTweets["FECHA"])
dfTweets.index = pd.to_datetime(dfTweets["FECHA"], unit='s')
dfTweets = dfTweets.sort_values('FECHA')
dfTweets = dfTweets[dfTweets["ID_CUENTA"]==1]

<div class="alert alert-block alert-info">
VARIABLE GLOBAL USADA PARA PODER PROBAR DIFERENTES AGRUPADOS DE TIEMPO DE LOS 3 TIPOS DE DATAFRAME:
<BR><BR>
- Transacciones de usuarios
<BR><BR>
- Transacciones de ballenas (>1.000.000)
<BR><BR>
- Tweets
</div> 

In [5]:
minutosDeAgrupado = 5

<div class="alert alert-block alert-info"> 
FUNCIONES PARA AGRUPAR LOS DATAFRAMES POR TIEMPO
<br><BR>
(Reciben como parámetro el dataframe y el número de minutos)
</div>

In [6]:
def agrupaTweets(df, minutos):
    
    df1 = df.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).mean()
    df1 = df1["SENTIMIENTO"]
    
    return df1

def agrupaBallenas(df, minutos):
    
    df['CANTIDAD_TOTAL'] = df["CANTIDAD"]
        
    df1 = df.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).count()
    df1["NUM_TRANSACCIONES"] = df1['CANTIDAD']
    df1 = df1["NUM_TRANSACCIONES"] 
    
    df2 = df.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).sum()
    df2 = df2['CANTIDAD_TOTAL']
    
    df = pd.concat([df1, df2],axis=1)
    
    return df

def agrupaDatos(df, minutos):
    
    #df = dfTransacciones
    dfCompras = df[df["TIPOTRANSACCION"]==False]
    dfVentas = df[df["TIPOTRANSACCION"]==True]

    # NUEVAS COLUMNAS
    df["HIGH"] = df["PRICE"]
    df['LOW'] = df["PRICE"]
    df['CANTIDAD_TOTAL'] = df["AMOUNT"]
    dfCompras['NUM_T_COMPRA'] = dfCompras["AMOUNT"]
    dfVentas['NUM_T_VENTA'] = dfVentas["AMOUNT"]

    # MAXIMO, MINIMO, CANTIDAD_TOTAL
    df1 = df.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).agg({'HIGH':np.max, 'LOW':np.min,'CANTIDAD_TOTAL':np.sum})

    # NÚMERO DE TRANSACCIONES DE COMPRA
    df2 = dfCompras.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).count()
    df2 = df2['NUM_T_COMPRA']

    # NÚMERO DE TRANSACCIONES DE VENTA
    df3 = dfVentas.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).count()
    df3 = df3['NUM_T_VENTA']

    # CANTIDAD TOTAL DE MONEDA COMPRADA
    df4 = dfCompras.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).sum() 
    df4["CANTIDAD_COMPRA"] = df4['AMOUNT']
    df4 = df4['CANTIDAD_COMPRA']

    # CANTIDAD TOTAL DE MONEDA VENDIDA
    df5 = dfVentas.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).sum()
    df5["CANTIDAD_VENTA"] = df5['AMOUNT']
    df5 = df5['CANTIDAD_VENTA']

    # OPEN
    df6 = df.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).first()
    df6["OPEN"] = df6["PRICE"]
    df6= df6["OPEN"]

    # CLOSE
    df7 = df.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).last()
    df7["CLOSE"] = df7["PRICE"]
    df7= df7["CLOSE"]

    df = pd.concat([df1, df2, df3, df4, df5, df6, df7], axis=1)

    # NUEVAS COLUMNAS CALCULADAS
    df['VAR_MAX'] = (df['HIGH'] - df['LOW'])/df['LOW'] * 100
    df['VAR_INTERVALO'] = (df['CLOSE'] - df['OPEN'])/df['OPEN'] * 100

    # FILTRAMOS COLUMNAS 
    df = df[['NUM_T_COMPRA','NUM_T_VENTA','CANTIDAD_COMPRA','CANTIDAD_VENTA','VAR_MAX','VAR_INTERVALO','OPEN', 'CLOSE']]

    df = df.dropna()
    
    return df

<div class="alert alert-block alert-info"> 
AGRUPAMOS LOS 3 DATAFRAMES
</div>

In [7]:
dfAgrupadoUsuarios = agrupaDatos(dfTransacciones, minutosDeAgrupado)
dfAgrupadoTweets = agrupaTweets(dfTweets, minutosDeAgrupado)
dfAgrupadoBallenas = agrupaBallenas(dfTransBallenas, minutosDeAgrupado)

<div class="alert alert-block alert-info"> 
CREAMOS DOS FEATURES NUEVAS PARA MEDIR LA TENSIÓN DEL MERCADO EN EL DATAFRME DE TRANSACCIONES
</div>


In [8]:
dfAgrupadoUsuarios["TENDENCIA_SUBIDA_ACUMULADA"] = 0 # 0-N FLOAT REGISTRA LA ACUMULACIÓN DE SUBIDA EN %
dfAgrupadoUsuarios["TENDENCIA_BAJADA_ACUMULADA"] = 0 # 0-N FLOAT REGISTRA LA ACUMULACIÓN DE BAJADA EN %

numColTendenciaSubida = 8
numColTendenciaBajada = 9

acumuladoSubida = 0
acumuladoBajada = 0
primeraSubida = True
primeraBajada = True

cont = 0

contDebugSub = 0
contDebugBaj = 0

for index, row in dfAgrupadoUsuarios.iterrows():
       
        # SI SUBE EL PRECIO
        if(row['VAR_INTERVALO'] > 0):      
            
            contDebugSub+=1
            
            if(primeraSubida):
                acumuladoBajada = 0
                primeraBajada = True
                primeraSubida = False
            
            acumuladoSubida += row['VAR_INTERVALO']
            dfAgrupadoUsuarios.iloc[cont,numColTendenciaSubida] = acumuladoSubida                                                
            
        # SI BAJA EL PRECIO
        else:             
            
            contDebugBaj+=1
            
            if(primeraBajada): 
                acumuladoSubida = 0
                primeraSubida = True
                primeraBajada = False
            
            acumuladoBajada += -row['VAR_INTERVALO']
            dfAgrupadoUsuarios.iloc[cont,numColTendenciaBajada] = acumuladoBajada                
           
        cont+=1
        

<div class="alert alert-block alert-info"> 
CREAMOS FEATURES NUEVAS EN FUNCIÓN DE LOS VALORES DE REGISTROS ANTERIORES
</div>

In [9]:
#########################################
######### VARIABLES ENTRADA #############
#########################################
numeroRegAnteriores = 5

dfFinal = dfAgrupadoUsuarios

for i in range(1, numeroRegAnteriores+1):
    dfAux =  dfAgrupadoUsuarios.shift(i)
    etiqueta = '_t(' + str(i) + ')'
    dfAux.columns = ['NUM_T_COMPRA' + etiqueta,
                    'NUM_T_VENTA' + etiqueta,
                    'CANTIDAD_COMPRA'  + etiqueta,
                    'CANTIDAD_VENTA' + etiqueta, 
                    'VAR_MAX' + etiqueta,
                    'VAR_INTERVALO' + etiqueta,
                    'OPEN' + etiqueta,
                    'CLOSE' + etiqueta,
                    'TENDENCIA_SUBIDA_ACUMULADA' + etiqueta,
                    'TENDENCIA_BAJADA_ACUMULADA' + etiqueta]
    dfFinal = pd.concat([dfFinal, dfAux], axis=1)


<div class="alert alert-block alert-info"> 
CREAMOS VARIABLE A PRONOSTICAR
<br><br>
Ya que buscamos probar aalgoritmos de clasificacion, estableceremos la variable como 1 en caso de subida, y 0 en caso de bajada.
</div>

In [10]:
numIntervarlosHorizonte = 3

mask = dfFinal["VAR_INTERVALO"].shift(-numIntervarlosHorizonte) > 0
dfFinal["PRONOSTICO"] = 0
dfFinal[mask]= 1

<div class="alert alert-block alert-success">
FASE 2 - FEATURES SELECTION
</div>

<div class="alert alert-block alert-info"> 
2.1 Univariate Selection
</div>

In [11]:
dfTemp = dfFinal.abs()

array = dfTemp.values
X = array[:,0:9]
X = min_max_scaler.fit_transform(X)
Y = array[:,9].astype('int')

test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, Y)
numpy.set_printoptions(precision=3)

print(fit.scores_)

[  242.916   775.723   281.988   335.157   323.72   1060.08   1394.58
  1376.461   472.757]


<div class="alert alert-block alert-info"> 
En esta primera selección de variables, 
</div>

<div class="alert alert-block alert-info"> 
2.2 Recursive Feature Elimination
</div>

In [12]:
dfTemp = dfFinal.abs()

array = dfTemp.values
X = array[:,0:9]
X = min_max_scaler.fit_transform(X)
Y = array[:,9].astype('int')

model = LogisticRegression()
rfe = RFE(model, 4)
fit = rfe.fit(X, Y)

print("Num Features:" , fit.n_features_)
print("Selected Features:", fit.support_)
print("Feature Ranking:", fit.ranking_)

Num Features: 4
Selected Features: [ True  True  True False False  True False False False]
Feature Ranking: [1 1 1 6 4 1 2 5 3]


<div class="alert alert-block alert-info"> 
2.3 Recursive Feature Elimination
</div>

In [13]:
dfTemp = dfFinal.abs()

array = dfTemp.values
X = array[:,0:9]
X = min_max_scaler.fit_transform(X)
Y = array[:,9].astype('int')

model = ExtraTreesClassifier()
model.fit(X, Y)
print(model.feature_importances_)

[ 0.017  0.043  0.033  0.021  0.228  0.304  0.15   0.12   0.084]


<div class="alert alert-block alert-success">
FASE 3 - MACHINE LEARNING
</div>

<div class="alert alert-block alert-info"> 
ML - FUNCIONES 
</div>

In [14]:
def calculaMetricas(y_test, y_pred):
    
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    return mae, mse, rmse
     
def mensajeAlgoritmo(start, nombre):
    
    segundos = time.time() - start
    if(segundos < 1):
          mensaje = nombre + " PROCESADO EN < 1 seg"
    else:
          mensaje = nombre + " PROCESADO EN " + str(time.time() - start) + " seg."              

    return mensaje


# ENUMERADOR DE ALGORITMOS
class modeloRegresion:
    
    # NaiveBayesClassifier
    SGDClassifier, DecisionTreeClassifier, GradientBoostingClassifier, RandomForest, KVecinos, XGBClassifier = range(6)
    
def train_test_split(df, nombreColumna, test_size=0.1):
    
    split_row = len(df) - int(test_size * len(df))
    train_data = df.iloc[:split_row]
    test_data = df.iloc[split_row:]
    
    x_ent = np.array(train_data.drop([nombreColumna],1)) 
    #x_ent = preprocessing.scale(x_ent)
    x_ent = preprocessing.robust_scale(x_ent)
    x_test = np.array(test_data.drop([nombreColumna],1))
    #x_test = preprocessing.scale(x_test)
    x_test = preprocessing.robust_scale(x_test)
    
    y_ent = np.array(train_data[nombreColumna]) 
    y_test = np.array(test_data[nombreColumna]) 
    
    return x_ent, x_test, y_ent, y_test

<div class="alert alert-block alert-info"> 
FUNCIONES DE ALGORITMOS DE MACHINE LEARNING
</div>

In [15]:
def obtenerModelo(df, nombreColumnaLabel,tipo):
    
    x_ent, x_test, y_ent, y_test = train_test_split(df,nombreColumnaLabel,test_size=0.2)    

    score = 0
    parametros = ''
    
    if(tipo==modeloRegresion.SGDClassifier): 
        
        nombre = "SGDClassifier"
        modelo = SGDClassifier() 
        
    if(tipo==modeloRegresion.DecisionTreeClassifier):                 
        nombre = "DecisionTreeClassifier"     
        
        # CALCULAMOS EL MEJOR MODELO EN FUNCION DEL MAXDEPTH   
        mejorParam1 = 0
        mejorScoreTree = 0
        
        for i in range(1,6):
            arbol = DecisionTreeClassifier(max_depth=i, criterion='gini')
            arbol.fit(x_ent,y_ent)
            score = arbol.score(x_test,y_test)

            if(score > mejorScoreTree):
                mejorScoreTree = score
                mejorParam1 = i        

        modelo = DecisionTreeClassifier(max_depth=mejorParam1, criterion='gini')
        parametros = 'max_depth:%s' %mejorParam1

    if(tipo==modeloRegresion.RandomForest): 
        
        nombre = "RandomForest"
        modelo = RandomForestClassifier(n_jobs=2, random_state=0)  
    
    
    if(tipo==modeloRegresion.GradientBoostingClassifier):                 
        
        nombre = "GradientBoostingClassifier"     
        
        # CALCULAMOS EL MEJOR MODELO EN FUNCION DEL MAXDEPTH   
        mejorParam1 = 0
        mejorScoreTree = 0
        
        for i in range(1,10):
            arbol = GradientBoostingClassifier(random_state=10,n_estimators=100, learning_rate=1.0, max_depth=i)
            arbol.fit(x_ent,y_ent)
            score = arbol.score(x_test,y_test)

            if(score > mejorScoreTree):
                mejorScoreTree = score
                mejorParam1 = i        
                
        modelo = GradientBoostingClassifier(max_depth=mejorParam1)
        parametros = 'max_depth:%s' %mejorParam1  
        
    if(tipo==modeloRegresion.KVecinos):                 
        
        nombre = "KVecinos"     
        
        # CALCULAMOS EL MEJOR MODELO EN FUNCION DEL NÜMERO DE VECINOS   
        mejorParam1 = 0
        mejorScoreTree = 0
        
        for i in range(2,8):
            aux = KNeighborsRegressor(n_neighbors=i)  
            aux.fit(x_ent,y_ent)
            score = aux.score(x_test,y_test)

            if(score > mejorScoreTree):
                mejorScoreTree = score
                mejorParam1 = i        
                
        modelo = KNeighborsRegressor(n_neighbors=mejorParam1)  
        parametros = 'max_depth:%s' %mejorParam1          
    
    if(tipo==modeloRegresion.XGBClassifier):                 
        
        nombre = "XGBClassifier"     
        modelo = xgb.XGBClassifier()  
        
        '''
        param = {}
        param['booster'] = 'gbtree'
        param['objective'] = 'binary:logistic'
        param["eval_metric"] = "error"
        param['eta'] = 0.3
        param['gamma'] = 0
        param['max_depth'] = 6
        param['min_child_weight']=1
        param['max_delta_step'] = 0
        param['subsample']= 1
        param['colsample_bytree']=1
        param['silent'] = 1
        param['seed'] = 0
        param['base_score'] = 0.5
        '''                    
        
    modelo.fit(x_ent,y_ent)    
        
    y_pred = modelo.predict(x_test)
    
    score = accuracy_score(y_test,y_pred)#*100
    #score = modelo.score(x_test,y_test)
    
    reportModelo = classification_report(y_test, y_pred)
 
    resultadoModelo = [nombre, score, parametros] 

    return resultadoModelo, reportModelo, x_ent, x_test, y_ent, y_test, y_pred


In [16]:
def obtenerModelosClasificacion(df, nombreColumnaLabel):
    
    dfResultados = pd.DataFrame(columns=('ALGORITMO','ACCURACY','PARÁMETROS'))  
    
    start = time.time()
    resultadoModelo, reportModelo, x_ent, x_test, y_ent, y_test, y_pred = obtenerModelo(dfFinal, "PRONOSTICO", modeloRegresion.SGDClassifier)
    dfResultados.loc[len(dfResultados)] = resultadoModelo
    print(reportModelo)
    print("PROCESADO SGDClassifier EN " + str(time.time() - start) + " segundos.")    
    
    start = time.time()
    resultadoModelo, reportModelo, x_ent, x_test, y_ent, y_test, y_pred = obtenerModelo(dfFinal, "PRONOSTICO", modeloRegresion.DecisionTreeClassifier)
    dfResultados.loc[len(dfResultados)] = resultadoModelo
    print(reportModelo)
    print("PROCESADO DecisionTree EN " + str(time.time() - start) + " segundos.")
    
    start = time.time()
    resultadoModelo, reportModelo, x_ent, x_test, y_ent, y_test, y_pred = obtenerModelo(dfFinal, "PRONOSTICO", modeloRegresion.RandomForest)
    dfResultados.loc[len(dfResultados)] = resultadoModelo
    print(reportModelo)
    print("PROCESADO RandomForest EN " + str(time.time() - start) + " segundos.")
    
    start = time.time()    
    resultadoModelo, reportModelo, x_ent, x_test, y_ent, y_test, y_pred = obtenerModelo(dfFinal, "PRONOSTICO", modeloRegresion.GradientBoostingClassifier)
    dfResultados.loc[len(dfResultados)] = resultadoModelo
    print(reportModelo)
    print("PROCESADO GradientBoostingClassifier EN " + str(time.time() - start) + " segundos.")
    
    start = time.time()    
    resultadoModelo, reportModelo, x_ent, x_test, y_ent, y_test, y_pred = obtenerModelo(dfFinal, "PRONOSTICO", modeloRegresion.RandomForest)
    dfResultados.loc[len(dfResultados)] = resultadoModelo
    print(reportModelo)
    print("PROCESADO RandomForest EN " + str(time.time() - start) + " segundos.")
    
    start = time.time()    
    resultadoModelo, reportModelo, x_ent, x_test, y_ent, y_test, y_pred = obtenerModelo(dfFinal, "PRONOSTICO", modeloRegresion.XGBClassifier)
    dfResultados.loc[len(dfResultados)] = resultadoModelo
    print(reportModelo)
    print("PROCESADO XGBClassifier EN " + str(time.time() - start) + " segundos.")
    
    return dfResultados


<div class="alert alert-block alert-info"> 
FUNCIONES DE VISUALIZACION
</div>

In [17]:
def pintaDatos(df):

    test_size=0.1
    split_row = len(df) - int(test_size * len(df))
    train_data = df.iloc[:split_row]
    test_data = df.iloc[split_row:]

    fig, ax = plt.subplots(1, figsize=(16, 9))
    ax.plot(train_data.CLOSE, label="DATOS DE ENTRENAMIENTO", linewidth=2)
    ax.plot(test_data.CLOSE, label="DATOS DE TEST", linewidth=2)
    ax.set_ylabel('PRECIO [XRP]', fontsize=14)
    ax.set_title("PREPARACION DE DATOS", fontsize=18)
    ax.legend(loc='best', fontsize=18)
    
def pintaPrecioPrediccion(df):
    
    split_row = len(dfFinal) - int(0.2 * len(dfFinal))
    train_data = dfFinal.iloc[:split_row]
    test_data = dfFinal.iloc[split_row:]

    fig, ax = plt.subplots(1, figsize=(16, 9))
    ax.plot(dfFinal.index, dfFinal["CLOSE"], label="PRECIO REAL", linewidth=2)
    ax.plot(test_data.index, test_data["CLOSE"] + y_pred * test_data["CLOSE"] / 100, label="PREDICCIÓN", linewidth=2)
    ax.set_ylabel('PRECIO [XRP]', fontsize=14)
    ax.set_title("COMPARACIÓN DE PRECIO REAL VS PREDICCIÓN", fontsize=18)
    ax.legend(loc='best', fontsize=18)

def pintaPorcentajes(y_test,y_pred):
    
    pylab.rcParams['figure.figsize'] = (30, 20)
    plt.plot(y_test,color='lightblue')
    plt.plot(y_pred)
    plt.show()

<div class="alert alert-block alert-info"> 
ENTRENAMIENTO / TEST
</div>

<div class="alert alert-block alert-info"> 
CON LA SIGUIENTE LLAMADA, PODEMOS OBTENER EL RESULTADO DE TODAS LAS MÉTRICAS ESTUDIADAS PARA CADA UNO DE LOS ALGORITMOS REGRESORES
</div>

In [18]:
obtenerModelosClasificacion(dfFinal,"PRONOSTICO")

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2429
          1       1.00      1.00      1.00      2161

avg / total       1.00      1.00      1.00      4590

PROCESADO SGDClassifier EN 0.26180005073547363 segundos.
             precision    recall  f1-score   support

          0       1.00      0.96      0.98      2429
          1       0.96      1.00      0.98      2161

avg / total       0.98      0.98      0.98      4590

PROCESADO DecisionTree EN 0.6359999179840088 segundos.
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2429
          1       1.00      1.00      1.00      2161

avg / total       1.00      1.00      1.00      4590

PROCESADO RandomForest EN 0.3380000591278076 segundos.
             precision    recall  f1-score   support

          0       1.00      0.96      0.98      2429
          1       0.96      1.00      0.98      2161

avg / total       0.98     

,ALGORITMO,ACCURACY,PARÁMETROS
0,SGDClassifier,1.000000,
1,DecisionTreeClassifier,0.978431,max_depth:1
2,RandomForest,0.999564,
3,GradientBoostingClassifier,0.978214,max_depth:1
4,RandomForest,0.999564,
5,XGBClassifier,0.978431,


<div class="alert alert-block alert-info"> 
MÉTODO AUXILIAR PARA EXPORTAR LOS DATOS A UN CSV PARA UTILIZAR EN D3
</div>

In [19]:
#dfAgrupadoUsuariosAux = agrupaDatos(dfTransacciones, 1440)
#dfAgrupadoCSV = dfAgrupadoUsuariosAux[['NUM_T_COMPRA','NUM_T_VENTA','CANTIDAD_COMPRA','CANTIDAD_VENTA']]
#dfAgrupadoCSV.to_csv('ficheroD3.csv', header=False, index=True)

In [21]:
#import csv
#import collections
#import pandas as pd

pathCSV = "ficheroD3.csv"

dfAgrupadoUsuariosAux = agrupaDatos(dfTransacciones, 1440)
dfAgrupadoCSV = dfAgrupadoUsuariosAux[['NUM_T_COMPRA','NUM_T_VENTA','CANTIDAD_COMPRA','CANTIDAD_VENTA']]

x = dfAgrupadoCSV.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
dfAgrupadoCSV = pd.DataFrame(x_scaled)
dfAgrupadoCSV = dfAgrupadoCSV*100
dfAgrupadoCSV = dfAgrupadoCSV.astype('int')
dfAgrupadoCSV.to_csv(pathCSV, header=False, index=True)